In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt


All exp are conducted on one H100

# Baseline CLIP Score

In [ ]:
baseline_datapoints = [
    # {
    #     "title": "baseline-with-cfg",
    #     "num_inference_steps": 0,
    #     "clip_score": 0.0,
    #     "throughput": 0.0,
    # },
    {
        "title": "baseline-with-cfg",
        "num_inference_steps": 2,
        "clip_score": 20.086650848388672,
        "throughput": 2.5590091626428144,
    },
    {
        "title": "baseline-with-cfg",
        "num_inference_steps": 4,
        "clip_score": 25.306303024291992,
        "throughput": 1.657429151534393,
    },
    {
        "title": "baseline-with-cfg",
        "num_inference_steps": 8,
        "clip_score": 31.721240997314453,
        "throughput": 0.9713379615909279,
    },
    {
        "title": "baseline-with-cfg",
        "num_inference_steps": 16,
        "clip_score": 33.564979553222656,
        "throughput": 0.5309598880016301,
    },
    {
        "title": "baseline-with-cfg",
        "num_inference_steps": 32,
        "clip_score": 33.70329284667969,
        "throughput": 0.2789380126607746,
    },
    {
        "title": "baseline-with-cfg",
        "num_inference_steps": 64,
        "clip_score": 33.596866607666016,
        "throughput": 0.14308945973509418,
    },
    {
        "title": "baseline-without-cfg",
        "num_inference_steps": 2,
        "clip_score": 20.119003295898438,
        "throughput": 3.933640387034316,
    },
    {
        "title": "baseline-without-cfg",
        "num_inference_steps": 4,
        "clip_score": 27.01666259765625,
        "throughput": 2.734942860113145,
    },
    {
        "title": "baseline-without-cfg",
        "num_inference_steps": 8,
        "clip_score": 30.11493492126465,
        "throughput": 1.7023422067884688,
    },
    {
        "title": "baseline-without-cfg",
        "num_inference_steps": 16,
        "clip_score": 31.239246368408203,
        "throughput": 0.9681250180951976,
    },
    {
        "title": "baseline-without-cfg",
        "num_inference_steps": 32,
        "clip_score": 31.468881607055664,
        "throughput": 0.5209463226809052,
    },
    {
        "title": "baseline-without-cfg",
        "num_inference_steps": 64,
        "clip_score": 31.545791625976562,
        "throughput": 0.2713344636771611,
    },

]

In [ ]:
# plot throughput-clip_score line
fig, ax = plt.subplots()

group_titles = [
    "baseline-with-cfg",
    "baseline-without-cfg",
]
title2datas = {
    title: [data for data in baseline_datapoints if data["title"] == title] for title in group_titles
}
for title in group_titles:
    datas = title2datas[title]
    throughputs = [data["throughput"] for data in datas]
    clip_scores = [data["clip_score"] for data in datas]
    ax.plot(throughputs, clip_scores, label=title)

ax.set_xlabel("Throughput (images/sec)")
ax.set_ylabel("Clip Score")
ax.legend()
ax.set_title("Throughput - Clip Score")
plt.show()

# Midj Distilled

In [ ]:
midj_datapoints = [
    # {
    #     "title": "midj-lr1e-5-with-cfg",
    #     "num_inference_steps": 0,
    #     "clip_score": 0.0,
    #     "throughput": 0.0,
    # },
    {
        "title": "midj-lr1e-5-with-cfg",
        "num_inference_steps": 2,
        "clip_score": 19.863466262817383,
        "throughput": 2.56941883763637,
    },
    {
        "title": "midj-lr1e-5-with-cfg",
        "num_inference_steps": 4,
        "clip_score": 21.4808292388916,
        "throughput": 1.666968143175522,
    },
    {
        "title": "midj-lr1e-5-with-cfg",
        "num_inference_steps": 8,
        "clip_score": 25.832571029663086,
        "throughput": 0.9795837993969497,
    },
    {
        "title": "midj-lr1e-5-with-cfg",
        "num_inference_steps": 16,
        "clip_score": 29.2199764251709,
        "throughput": 0.5351153861167683,
    },
    {
        "title": "midj-lr1e-5-with-cfg",
        "num_inference_steps": 32,
        "clip_score": 31.524333953857422,
        "throughput": 0.2806375074234804,
    },
    {
        "title": "midj-lr1e-5-with-cfg",
        "num_inference_steps": 64,
        "clip_score": 33.00955581665039,
        "throughput": 0.14436954344316758,
    },
    {
        "title": "midj-lr1e-5-without-cfg",
        "num_inference_steps": 2,
        "clip_score": 22.69116973876953,
        "throughput": 3.930560550569773,
    },
    {
        "title": "midj-lr1e-5-without-cfg",
        "num_inference_steps": 4,
        "clip_score": 32.02427291870117,
        "throughput": 2.730854775844102,
    },
    {
        "title": "midj-lr1e-5-without-cfg",
        "num_inference_steps": 8,
        "clip_score": 32.738929748535156,
        "throughput": 1.698274155853291,
    },
    {
        "title": "midj-lr1e-5-without-cfg",
        "num_inference_steps": 16,
        "clip_score": 32.446815490722656,
        "throughput": 0.9656715471290394,
    },
    {
        "title": "midj-lr1e-5-without-cfg",
        "num_inference_steps": 32,
        "clip_score": 32.1515007019043,
        "throughput": 0.5193002900222913,
    },
    {
        "title": "midj-lr1e-5-without-cfg",
        "num_inference_steps": 64,
        "clip_score": 31.964906692504883,
        "throughput": 0.2696665824969441,
    },

]

In [ ]:
# plot throughput-clip_score line
fig, ax = plt.subplots()
import itertools
group_titles = [
    "midj-lr1e-5-with-cfg",
    "midj-lr1e-5-without-cfg",
    "baseline-with-cfg",
    "baseline-without-cfg",
]
title2datas = {
    title: [data for data in itertools.chain(midj_datapoints, baseline_datapoints) if data["title"] == title] \
        for title in group_titles
}
for title in group_titles:
    datas = title2datas[title]
    throughputs = [data["throughput"] for data in datas]
    clip_scores = [data["clip_score"] for data in datas]
    ax.plot(throughputs, clip_scores, label=title)

ax.set_xlabel("Throughput (images/sec)")
ax.set_ylabel("Clip Score")
ax.legend()
ax.set_title("Throughput - Clip Score")
plt.show()